# Линейная регрессия

Данные содержатся в файле `data/lin_reg.txt`. Прочитать их можно следующим так:

In [1]:
import numpy as np
D = np.loadtxt('../data/lin_reg.txt', delimiter=',')
X = D[:, :-1]
Y = D[:, -1]

При выполнении заданий **нельзя** пользоваться `scikit-learn` и т.п., только `NumPy`/`JAX`/`TF`/`PyTorch` и `matplotlib`/`seaborn`/`plotly`/etc.

1. Нарисуйте график среднеквадратичной ошибки в зависимости от параметра регуляризации $\alpha$, используя полные данные для обучения и для тестирования. 
2. Подготовьте исходные данные для 5 fold CV.
3. С помощью CV найдите оптимальное значение $\alpha$ на какой-нибудь сетке значений.
4. Постройте валидационную кривую. 
5. Постройте кривую обучения.

In [2]:
X, Y

(array([[  9.31 , -17.043,  20.807,  11.109],
        [ -6.446, -12.46 , -12.941, -14.715],
        [ 14.324,  -9.624,  29.533,   1.439],
        ...,
        [  3.56 ,   2.063,   7.302,  -7.817],
        [ 11.841, -14.315,  23.148,  11.777],
        [ -3.345, -15.219,  -4.751,  -1.141]]),
 array([-165.864,  -49.048, -161.183,  177.828,   40.092,  132.818,
        -210.962,  -11.84 ,  181.097,  -28.715,  -85.778,   55.17 ,
         121.387,   79.768,   53.727,   45.013,  -41.435,  -91.088,
         129.3  ,  138.069,  -32.319,  -93.994,   54.087,  -18.257,
          78.758, -105.918,   60.61 , -113.481,  122.498, -127.546,
          86.298,   92.545,  144.638,  -23.472,   75.777,  241.73 ,
         124.678,  160.016, -218.938,  -13.655,   80.752,   32.972,
        -120.015,  -52.363,  109.309,  -51.847,   12.276,  193.39 ,
         200.685,  215.559,  169.829,   16.579,  145.243, -204.876,
          78.312,  -10.712, -144.72 ,   82.893,  -13.148,  159.075,
        -185.267,   84.177,  